In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import requests as re
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

### Reading in TN County shapes

In [ ]:
counties = gpd.read_file(r"C:\Users\jared\Desktop\nss\capstone\data\TIGER_Line_2018_Tennessee_Counties.geojson")
counties.head(5)

### All dollar stores df

In [ ]:
dollar_stores = pd.read_csv(r'C:\Users\jared\Desktop\nss\capstone\data\dollar_stores.csv')
dollar_stores

In [ ]:
dollar_stores = dollar_stores.drop(columns = ['Unnamed: 0'])
dollar_stores

### Giving dollar_stores a geometry column and converting to geodataframe

In [ ]:
dollar_stores['geometry'] = dollar_stores.apply(lambda x: Point((float(x.Longitude), 
                                                         float(x.Latitude))), 
                                        axis=1)
dollar_stores.head(3)

In [ ]:
dollar_stores_geo = gpd.GeoDataFrame(dollar_stores, 
                           crs = counties.crs, 
                           geometry = dollar_stores['geometry'])

In [ ]:
type(dollar_stores_geo)

In [ ]:
counties = counties[['NAMELSAD','geometry']]
counties

### Geospatial join

In [ ]:
dollar_stores_by_county = gpd.sjoin(dollar_stores_geo, counties, op = 'within')

In [ ]:
dollar_stores_by_county

### Count of dollar stores by county

In [ ]:
count_by_county = dollar_stores_by_county['NAMELSAD'].value_counts()
count_by_county

count_by_county = pd.DataFrame(count_by_county)

In [ ]:
count_by_county.tail(5)

### Merging count per county column onto dollar stores df

In [ ]:
Count_Merge = pd.merge(dollar_stores_by_county, count_by_county,  how='inner', left_on=['NAMELSAD'], right_on = ['NAMELSAD'])
Count_Merge

In [ ]:
Count_Merge = Count_Merge.rename(columns = {'NAMELSAD':'County'})

In [ ]:
Count_Merge

In [ ]:
Count_Merge['County'] = Count_Merge['County'].replace('DeKalb County','De Kalb County')

### Importing, cleaning, and joining metrics to dollar store locations df

In [ ]:
obesity = pd.read_csv(r'C:\Users\jared\Desktop\nss\capstone\data\DiabetesAtlas_CountyData.csv')
obesity

In [ ]:
obesity = obesity.drop(columns = ["State","CountyFIPS","Lower Limit"," Upper Limit"])

In [ ]:
obesity['County'] = obesity['County'].replace('Dekalb County','De Kalb County')
obesity['County'] = obesity['County'].replace('Mcnairy County','McNairy County')
obesity['County'] = obesity['County'].replace('Mcminn County','McMinn County')

In [ ]:
obesity

In [ ]:
metrics = pd.read_csv(r'C:\Users\jared\Desktop\nss\capstone\data\all_metrics.csv')
metrics

In [ ]:
metrics['County'] = metrics['County'].str.replace(', Tennessee','')
metrics

In [ ]:
metrics = metrics.drop(columns = ["Unnamed: 0","YEAR"])

In [ ]:
metrics.loc[metrics['County'] == 'DeKalb County']

In [ ]:
metrics['County'] = metrics['County'].replace('DeKalb County','De Kalb County')

In [ ]:
metrics.loc[metrics['County'] == 'De Kalb County']

In [ ]:
metrics

In [ ]:
Food_Insecurity = pd.read_csv(r"C:\Users\jared\Desktop\nss\capstone\data\food insecurity.csv")


In [ ]:
Food_Insecurity = Food_Insecurity.drop(columns = ['# Food Insecure','# Limited Access to Healthy Foods','% Limited Access to Healthy Foods'])

In [ ]:
Food_Insecurity

### Merging clean metrics onto dollar store locations df

In [ ]:
Count_Merge = pd.merge(Count_Merge, metrics,  how='left', left_on=['County'], right_on = ['County'])
Count_Merge

In [ ]:
Count_Merge = pd.merge(Count_Merge, obesity,  how='left', left_on=['County'], right_on = ['County'])
Count_Merge

In [ ]:
Count_Merge = pd.merge(Count_Merge, Food_Insecurity,  how='left', left_on=['County'], right_on = ['County'])
Count_Merge

In [ ]:
Count_Merge = Count_Merge.drop(columns = ["index_right"])

In [ ]:
Count_Merge = Count_Merge.rename(columns = {'count':'County_Store_Count', 'Median_Income':'hh_med_inc','Percentage':'Obesity_Rate'})

In [ ]:
Count_Merge['Stores_Per_Capita'] = Count_Merge['County_Store_Count'] / Count_Merge['Population']
Count_Merge.sort_values(by = 'Stores_Per_Capita', ascending = False)

In [ ]:
Count_Merge['Stores_Per_10k'] = Count_Merge['Stores_Per_Capita'] * 10000
Count_Merge.sort_values(by = "Stores_Per_10k", ascending = False)

## Exploratory Data Analysis

In [ ]:
count_by_county = count_by_county.reset_index()

In [ ]:
count_by_county = count_by_county.rename(columns = {'NAMELSAD':'County','count':'Dollar Stores'})

In [ ]:
count_by_county_w_metrics = pd.merge(count_by_county, metrics,  how='inner', left_on=['County'], right_on = ['County'])

In [ ]:
count_by_county_w_metrics

In [ ]:
count_by_county_w_metrics = pd.merge(count_by_county_w_metrics, obesity,  how='left', left_on=['County'], right_on = ['County'])

In [ ]:
count_by_county_w_metrics = pd.merge(count_by_county_w_metrics, Food_Insecurity,  how='left', left_on=['County'], right_on = ['County'])

In [ ]:
count_by_county_w_metrics

In [ ]:
one_metric = Count_Merge[['County','Stores_Per_10k']]
one_metric

In [ ]:
one_metric = one_metric.groupby(one_metric['County']).mean()
one_metric

### Count of dollar stores by County w all metrics

In [ ]:
count_by_county_w_metrics = pd.merge(count_by_county_w_metrics, one_metric,  how='left', left_on=['County'], right_on = ['County'])
count_by_county_w_metrics

In [ ]:
count_by_county_w_metrics = count_by_county_w_metrics.rename(columns = {'Median_Income':'hh_med_inc','Percentage':'Obesity_Rate'})

### Top 5 counties with most dollar stores, correlated to population

In [ ]:
count_by_county_w_metrics.sort_values(by = "Dollar Stores", ascending = False).head(5)

### Top 5 counties with most dollar stores relative to population

In [ ]:
count_by_county_w_metrics.sort_values(by = "Stores_Per_10k", ascending = False).head(5)

In [ ]:
count_by_county_w_metrics.sort_values(by = "Stores_Per_10k", ascending = False).tail(5)

In [ ]:
metrics_corr = count_by_county_w_metrics.drop(columns =['County']).corr()

In [ ]:
metrics_corr

### Brands

In [ ]:
Count_Merge['store'].value_counts().plot(kind = 'bar',
                                        figsize = (10,6))                         

plt.xticks(rotation = 0,                                                
           fontsize = 12)                   
plt.title('Number of Dollar Stores by Brand',                              
         fontsize = 14,
         fontweight = 'bold');

In [ ]:
ct = pd.crosstab(Count_Merge['County'], Count_Merge['store'])
ct.reset_index()

### .526 correlation between Food Insecurity and Stores per 10,000 people

In [ ]:
plt.scatter(data = count_by_county_w_metrics, x = '% Food Insecure', y = 'Stores_Per_10k')
plt.xlabel('% Food Insecure')
plt.ylabel('Stores_Per_10k')

### -0.553 correlation between household median income and Stores per 10,000 people

In [ ]:
plt.scatter(data = count_by_county_w_metrics, x = 'hh_med_inc', y = 'Stores_Per_10k')
plt.xlabel('hh_med_inc')
plt.ylabel('Stores_Per_10k')

### -0.229 correlation between Obesity Rate and Stores per 10,000 people

In [ ]:
plt.scatter(data = count_by_county_w_metrics, x = 'Obesity_Rate', y = 'Stores_Per_10k')
plt.xlabel('Obesity_Rate')
plt.ylabel('Stores_Per_10k')

### 0.413 correlation between Poverty Rate and Stores per 10,000 people

In [ ]:
plt.scatter(data = count_by_county_w_metrics, x = 'Poverty_Rate', y = 'Stores_Per_10k')
plt.xlabel('Poverty_Rate')
plt.ylabel('Stores_Per_10k')

### More EDA

In [ ]:
Count_Merge.loc[Count_Merge['County'] == 'McMinn County']

In [ ]:
#Count_Merge.to_csv(r'C:\Users\jared\Desktop\nss\capstone\data\dollar_stores_geo_2.csv', index=False)

In [ ]:
#count_by_county_w_metrics.to_csv(r'C:\Users\jared\Desktop\nss\capstone\data\count_by_county_geo_2.csv', index=False)

In [ ]:
Count_Merge

In [ ]:
ct

In [ ]:
ct['Highest_Store_Count'] = ct.idxmax(axis=1)
ct

In [ ]:
maxes = pd.merge(Count_Merge, ct,  how='left', left_on=['County'], right_on = ['County'])
maxes

In [ ]:
#maxes.to_csv(r'C:\Users\jared\Desktop\nss\capstone\data\maxes.csv', index=False)